Question 2

In [1]:

'''
Pick any image frame from the 10 sec video footage.
Pick a region of interest in the image making sure there is a CORNER in that region.
Pick a 5 x 5 image patch in that region that constitutes the edge. 
Perform the steps of HARRIS CORNER DETECTION manually and note the pixels that correspond to the CORNER. 
Compare the outcome with MATLAB or OpenCV or DepthAI’s  Harris corner detection function.
'''

In [2]:
# Importing necessary libraries
import cv2
import depthai as dai
import numpy as np

import time
from pathlib import Path
from matplotlib import pyplot as plt

In [4]:
# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a source - left grayscale cameras
cam_left = pipeline.createMonoCamera()
cam_left.setBoardSocket(dai.CameraBoardSocket.CAM_B)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

# Create outputs
xout_left = pipeline.createXLinkOut()
xout_left.setStreamName('left')
cam_left.out.link(xout_left.input)

In [6]:
## RECORDING FOR 10 SECONDS
## THE FRAMES ARE BEING STORED IN DIRECTORY "video_frames"

# Connect and start the pipeline
with dai.Device(pipeline) as device:

    # Output queue will be used to get the grayscale frames from the output defined above
    q = device.getOutputQueue(name="left", maxSize=4, blocking=False)

    # Make sure the destination path is present before starting to store the examples
    Path(f"video_frames/").mkdir(parents=True, exist_ok=True)

    # running loop for 10 secs
    ten_secs = time.time() + 10
    
    while time.time() < ten_secs:
        # Blocking call, will wait until a new data has arrived
        inSrc = q.get()  
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        frame = inSrc.getCvFrame()
        # Frame is transformed and ready to be shown
        cv2.imshow("left", frame)
        cv2.waitKey(1)

        cv2.imwrite(f"video_frames/{int(time.time() * 10000)}.png", frame)

    cv2.destroyAllWindows()            

In [5]:
# harris conner detection
# choosing a random image from the captured frames
image = cv2.imread('video_frames/17109832419099.png')
start_row = 100
start_col = 150

# Extract the patch
image_patch = image[start_row:start_row+5, start_col:start_col+5]


# display input image
cv2.imshow('Image input', image_patch)
cv2.waitKey(1000)
cv2.destroyAllWindows()            

# converting to gray-scale
bw_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
bw_image = np.float32(bw_image)

# applying harris corner detection algorithm
dest = cv2.cornerHarris(bw_image, 2, 5, 0.07)
 
# Results are marked through the dilated corners
dest = cv2.dilate(dest, None)
 
# Reverting back to the original image,
# with optimal threshold value
image[dest > 0.01 * dest.max()]=[0, 0, 255]
 
# Image with corners marked 
# The image is stored in ______ for future references
cv2.imshow('Image with Borders', image)
cv2.waitKey(1000)
cv2.imwrite("harris_corner_detection.png", image)
cv2.destroyAllWindows()